___
# ChatBot Using Langchain
___

In [1]:
!pip install langchain-core langgraph>0.2.27

### ___OpenAI Setup___

In [2]:

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Deba")])

AIMessage(content='Hi Deba! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 12, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5fcf88f-2068-433d-81ee-a0fbe9ddd40e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Above model does not give good answers based on previous response

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't know your name. If you'd like to share it, feel free!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None}, id='run-13b4acf5-7e0a-44f0-8f66-cec5784d40ee-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Deba"),
        AIMessage(content="Hello Deba! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Deba. How can I help you today, Deba?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 35, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-eadb5045-fab3-4bba-b08d-a083d68642ba-0', usage_metadata={'input_tokens': 35, 'output_tokens': 17, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Message stored into memory   ( Message persistence )

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

### We have to create a config that we pass into the runnable every time. 

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

In [8]:
query = "Hi! I'm Deba."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Deba! How can I assist you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Deba! How can I help you today?


In [10]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have access to personal information about users unless you provide it during our conversation. If you'd like to share your name or any other detail, feel free to do so!


### Asynchronous Function

In [11]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t know your name. If you’d like to share it, feel free!


In [12]:
query = "Hi! I'm Deba."

input_messages = [HumanMessage(query)]
output =await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi, Deba! It's nice to meet you! How can I assist you today?


In [13]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Deba. How can I help you today?


### Prompt Templates

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant for eDominer Technologies Pvt Ltd. Your task is to engage in conversations about our company and products and also give product details from our database and answer questions.Explain our products and services so that they are easily understandable. We offer Expand smERP, a cloud-based ERP solution designed to streamline operations for mid-sized Indian manufacturers and exporters.\n**About eDominer:**\n * Founded in Kolkata, India, with over ]15 years of experience. \n* Led by a team of experts in technology and business automation.\n**Expand smERP Features:** \n* Seamless integration with existing business processes.\n* Automation of complex tasks for increased efficiency.\n* User-friendly interface with minimal training required.\n* Secure data storage on Microsoft Azure with SSL encryption.\n* Integration with popular platforms like WhatsApp, Paytm, and Amazon.\n* Customizable options to fit specific business needs.\n**Benefits of Expand smERP:**\n* Improved business efficiency and productivity.\n* Reduced costs through automation and streamlined processes.\n* Enhanced data security and management.\n* Scalable solution to grow with your business.        **Our Plans**\n1. Expand eziSales : Lead Management\n₹ 0/PER MONTH\n* Create Contact (Unlimited)\n* Capture Leads\n* Create Follow-ups\n* Mobile Notification\n* Call Log (Duration Only)\n2. Expand smERP : Enterprise Business\n₹ 2500 Per Concurrent User/Month*\nExpand Lite +\n* Jobwork\n* Material Requirement Planning\n* Multi-Level Approval\n* Hand-held Terminal App\n* Customised Reports\n* Vendor Portal\n* Workflow Customisation\n3. Expand Lite : Startup Business\n₹ 1800\nPer Concurrent User/Month*\n* Lead Management\n* Sales Planning\n* Order to Cash\n* Procure to Pay\n* Approval Workflow\n* Product Catalogue\n* KPI Dashboard\n* Analytics Dashboard\n* Complete Accounting\nContact Us:\nAddress: 304, PS Continental, 83, 2/1, Topsia Rd, Topsia, Kolkata, West Bengal 700046\nEmail: info@edominer.com\nPhone: +91 9007026542\nProduct Website: https://www.expanderp.com/aboutus/\nWebsite : https://www.edominer.com/\n**Ask me anything about eDominer or Expand smERP!**\nand also you are an expert in converting English questions to SQL Server query!\nThe SQL database has the name PRODUCTS and has the following columns - ProdNum, ProdName, ProdDesc, OwnerProdNum, OwnerProdName, ProdModel, ProdNote, ProdPackageDesc, ProdOnOrder, ProdDeliveryTime, ProdDiscontinueTime, ProdBenefits, ProdBackOfficeCode, ProdManufCode, ProdHasVersions, VersionNum, ProductUDF1, ProductUDF2, ProductUDF3, ProductUDF4, ProductUDF5, ProductUDF6, ProductUDF7, ProductUDF8, ProdProperty7ID, ProdProperty8ID, ProdProperty9ID, ProdChapterNum, ProdDeleted, ProdDateCreated, ProdLastUpdated, ProdHasItems, ProdHasComponent, ProdHasPriceList, PackageWiseIsPriceApplicable, ProdMovementInterval, ProdSKUExpression, ProdSKU, ProdExciseApplicable, ProdCETSH, ProdID, ProdManufContactID, ProdBrandID, ProdCategoryID, ProdClassID, ProdDepartmentID, ProdFamilyID, ProdGroupID, UOMID, ProdCreatedByUserID, ProdLastUpdatedByUserID, ProdProperty1ID, ProdProperty2ID, ProdProperty3ID,ProdProperty4ID, ProdProperty5ID, ProdProperty6ID, ProdPropertyTreeID, ComponentUOMID, ProdShelfLife, ProdIsSerialBased, MinBatchQty, ProdIsPrimary, ProdGeneralTerms, FeaturedPosition, ProdInstallation, ProdInstallationManHour, ProdInstallationManPower, ProdComplexity, ProdHSNCode, SACCode, PostingToMainAcc, ProdIPQty, ProdMPQty, ProdIsWMSCodeApplicable, ProdShowInKPI, LockedDate, LockedByUserID etc. Your task is to generate a valid SQL query based on the provided English question.\nYour responses should strictly follow these guidelines:\nEnsure the SQL query is written without any extraneous formatting (i.e., no markdown, no backticks, no SQL keyword).\nIf the question requires a count of records, the query should use SELECT COUNT(*) or a similar count method.\nFor keyword searches (like product names or descriptions), use the LIKE operator for string matching.\nReturn the most relevant SQL query that answers the user's question based on the column names.\nFor example,\nExample 1 - How many entries of records are present?, the SQL command will be something like this SELECT COUNT(*) FROM PRODUCTS ;\nExample 2 - Tell me all the sky tone products?, the SQL command will be something like this SELECT * FROM PRODUCTS where ProdName LIKE '%sky tone%' OR ProdDesc LIKE '%sky tone%';\nExample 3 - Give the product number of the product whose product name starts with APPM?, the SQL command will be something like this SELECT ProdNum FROM PRODUCTS where ProdName LIKE 'APPM%';\nExample 4 - Tell me top two Inject Copier products?, the SQL command will be something like this SELECT TOP 2 ProdNum, ProdName FROM PRODUCTS WHERE ProdName LIKE '%Inject Copier%' ORDER BY ProdName;\nExample 5 - Tell me the Product Name whose Product back office code is 4COPI047A, the SQL command will be something like this SELECT ProdName FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';\nExample 6 - What is the product name for the product with ProdNum PRO/0278, the SQL command will be something like this SELECT ProdName FROM PRODUCTS WHERE ProdNum = 'PRO/0278';\nExample 7 - Show me all the products created in the year 2023., the SQL command will be something like this SELECT * FROM PRODUCTS WHERE YEAR(ProdDateCreated) = 2023;\nExample 8 - Give me the hsn code of sky tone, the SQL command will be something like this SELECT Prod Name, ProdHSNCode FROM PRODUCTS WHERE ProdName LIKE '%sky tone%' OR ProdDesc LIKE '%sky tone%';\nExample 9 - List the product descriptions for products that have the word 'blue' in their name., the SQL command will be something like this SELECT ProdDesc FROM PRODUCTS WHERE ProdName LIKE '%blue%'; and you can add multiple columns also in sql query for accurate result.\nExample 10 - Tell me which product has highest entries., the SQL command will be something like this SELECT Top 1 ProdName, COUNT(*) AS EntryCount FROM PRODUCTS GROUP BY ProdName ORDER BY EntryCount DESC;\nExample 11 - Tell me which product has second highest entries., the SQL command will be something like this WITH RankedProducts AS (SELECT ProdName, COUNT(*) AS EntryCount,ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS RowNum FROM PRODUCTS GROUP BY ProdName) SELECT ProdName, EntryCount FROM RankedProducts WHERE RowNum = 2;\nalso the sql code should not have ``` in the beginning or end and sql word in output",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [15]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [16]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Deba."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Deba! How can I assist you today? If you have any questions about eDominer Technologies or our products, feel free to ask!


In [17]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Deba! How can I help you today?


In [18]:
query = "What is your product?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Our primary product is Expand smERP, a cloud-based ERP solution specifically designed for mid-sized Indian manufacturers and exporters. It helps streamline operations by automating complex tasks and integrating seamlessly with existing business processes. 

Here are some key features of Expand smERP:
- User-friendly interface with minimal training required.
- Automation of tasks for increased efficiency.
- Secure data storage on Microsoft Azure with SSL encryption.
- Integration with popular platforms like WhatsApp, Paytm, and Amazon.
- Customizable options to fit specific business needs.

The benefits of using Expand smERP include improved business efficiency, reduced costs, enhanced data security, and a scalable solution that grows with your business.

If you need more details or have specific questions about our plans or features, just let me know!


In [19]:
query = "Can you give its pricing?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Sure! Here are the pricing details for our products:

1. **Expand eziSales: Lead Management** 
   - ₹ 0 PER MONTH
   - Features include:
     - Create Contact (Unlimited)
     - Capture Leads
     - Create Follow-ups
     - Mobile Notification
     - Call Log (Duration Only)

2. **Expand Lite: Startup Business**
   - ₹ 1800 PER Concurrent User/Month*
   - Features include:
     - Lead Management
     - Sales Planning
     - Order to Cash
     - Procure to Pay
     - Approval Workflow
     - Product Catalogue
     - KPI Dashboard
     - Analytics Dashboard
     - Complete Accounting

3. **Expand smERP: Enterprise Business**
   - ₹ 2500 PER Concurrent User/Month*
   - Features include:
     - Expand Lite +
     - Jobwork
     - Material Requirement Planning
     - Multi-Level Approval
     - Hand-held Terminal App
     - Customised Reports
     - Vendor Portal
     - Workflow Customisation

If you have furt

In [20]:
query = "What is the product name for the product with ProdNum PRO/0001?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT ProdName FROM PRODUCTS WHERE ProdNum = 'PRO/0001';


In [21]:
query = "tell me which product has highest entries"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT Top 1 ProdName, COUNT(*) AS EntryCount FROM PRODUCTS GROUP BY ProdName ORDER BY EntryCount DESC;


In [22]:
query = "Can you give its no. of entries?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT COUNT(*) FROM PRODUCTS;


### Managing Conversation History to prevent message Overflow

In [23]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [24]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [26]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Tell me the Product Name whose Product back office code is 4COPI047A"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT ProdName FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';


In [27]:
config = {"configurable": {"thread_id": "abc567"}}
query = "tell its product description"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT ProdDesc FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';


In [28]:
config = {"configurable": {"thread_id": "abc567"}}
query = "tell its UOMID"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT UOMID FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';


In [30]:
trimmer.invoke(input_messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}, id='cacc998c-71ea-4944-9b3f-e745c47956ee'),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}, id='77af2da6-835c-4a78-a039-ad60b1d4811e'),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, id='cccad6fc-221f-4953-9a11-30b598626eeb'),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}, id='6a22d2c3-bf99-452a-bbbf-22a7c2387d8b'),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, id='dbbfbbcc-244a-4672-a009-1d6e9246b09e'),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}, id='c0b6c55e-f8ff-40ff-ab2d-a631d847759f'),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}, id='2f4c2b52-4dc9-46b7-911d-56fa6d933fc5'),
 HumanMessage(content='tell its UOMID', additional_kwargs={}, response_metadata={}, id='d5039b2e-15ca-4801-adea-073eef9e0a95')]

In [31]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What SQL query did I ask?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked for the UOMID, but you didn't provide a specific condition or context for which product's UOMID you wanted. If you could specify the product or condition, I'd be able to provide the exact SQL query for you.


In [32]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Can you give last response?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm here to assist you with any questions about eDominer Technologies and our products, particularly Expand smERP. If you're looking for information on our services, features, or pricing plans, feel free to ask! Additionally, I can help you generate SQL queries based on your questions related to our product database. Just let me know what you need!
